### Getting Started

You will notice that throughout this course, there is a lot of context switching between PySpark/Scala and SQL.

This is because:
* `read` and `write` operations are performed on DataFrames using PySpark or Scala
* table creates and queries are performed directly off Delta Lake tables using SQL

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Key Concepts: Delta Lake Architecture</h2>

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We'll touch on this further in future notebooks.

Throughout our Delta Lake discussions, we'll often refer to the concept of Bronze/Silver/Gold tables. These levels refer to the state of data refinement as data flows through a processing pipeline.

**These levels are conceptual guidelines, and implemented architectures may have any number of layers with various levels of enrichment.** Below are some general ideas about the state of data in each level.

* **Bronze** tables
  * Raw data (or very little processing)
  * Data will be stored in the Delta format (can encode raw bytes as a column)
* **Silver** tables
  * Data that is directly queryable and ready for insights
  * Bad records have been handled, types have been enforced
* **Gold** tables
  * Highly refined views of the data
  * Aggregate tables for BI
  * Feature tables for data scientists

For different workflows, things like schema enforcement and deduplication may happen in different places.

## Delta Lake Batch Operations - Create

Creating Delta Lakes is as easy as changing the file type while performing a write. 

In this section, we'll read from a CSV and write to Delta.

![](https://files.training.databricks.com/images/adbcore/AAFxQkg_SzRC06GvVeatDBnNbDL7wUUgCg4B.png)

Read the data into a DataFrame. We supply the schema.

Use overwrite mode so that there will not be an issue in rewriting the data in case you end up running the cell again.

Partition on `Country` because there are only a few unique countries and because we will use `Country` as a predicate in a `WHERE` clause.

More information on the how and why of partitioning is contained in the links at the bottom of this notebook.

Then write the data to Delta Lake.

In [ ]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

rawDataDF = (spark.read
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath)
)

# write to Delta Lake
rawDataDF.write.mode("overwrite").format("delta").partitionBy("Country").save(DataPath)

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> While we show creating a table in the next section, Spark SQL queries can run directly on a directory of data, for delta use the following syntax: 
```
SELECT * FROM delta.`/path/to/delta_directory`
```

In [ ]:
display(spark.sql("SELECT * FROM delta.`{}` LIMIT 5".format(DataPath)))

### CREATE A Table Using Delta Lake

Create a table called `customer_data_delta` using `DELTA` out of the above data.

The notation is:
> `CREATE TABLE <table-name>` <br>
  `USING DELTA` <br>
  `LOCATION <path-do-data> ` <br>
  
Tables created with a specified `LOCATION` are considered unmanaged by the metastore. Unlike a managed table, where no path is specified, an unmanaged table’s files are not deleted when you `DROP` the table. However, changes to either the registered table or the files will be reflected in both locations.

<img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> Managed tables require that the data for your table be stored in DBFS. Unmanaged tables only store metadata in DBFS. 

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Since Delta Lake stores schema (and partition) info in the `_delta_log` directory, we do not have to specify partition columns!

In [ ]:
spark.sql("""
  DROP TABLE IF EXISTS customer_data_delta
""")
spark.sql("""
  CREATE TABLE customer_data_delta
  USING DELTA
  LOCATION '{}'
""".format(DataPath))

Perform a simple `count` query to verify the number of records.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Notice how the count is right off the bat; no need to worry about table repairs.

In [ ]:
%sql
SELECT count(*) FROM customer_data_delta

### Metadata

Since we already have data backing `customer_data_delta` in place,
the table in the Hive metastore automatically inherits the schema, partitioning,
and table properties of the existing data.

Note that we only store table name, path, database info in the Hive metastore,
the actual schema is stored in the `_delta_log` directory as shown below.

In [ ]:
display(dbutils.fs.ls(DataPath + "/_delta_log"))

Metadata is displayed through `DESCRIBE DETAIL <tableName>`.

As long as we have some data in place already for a Delta Lake table, we can infer schema.

In [ ]:
%sql
DESCRIBE DETAIL customer_data_delta

### Key Takeaways

Saving to Delta Lake is as easy as saving to Parquet, but creates an additional log file.

Using Delta Lake to create tables is straightforward and you do not need to specify schemas.

## Delta Lake Batch Operations - Append

In this section, we'll load a small amount of new data and show how easy it is to append this to our existing Delta table.

We'll start start by setting up our relevant path and loading new consumer product data.

In [ ]:
miniDataInputPath = "/mnt/training/online_retail/outdoor-products/outdoor-products-mini.csv"

newDataDF = (spark
  .read
  .option("header", "true")
  .schema(inputSchema)
  .csv(miniDataInputPath)
)

Do a simple count of number of new items to be added to production data.

### APPEND Using Delta Lake

Adding to our existing Delta Lake is as easy as modifying our write statement and specifying the `append` mode. 

Here we save to our previously created Delta Lake at `delta/customer-data/`.

In [ ]:
(newDataDF
  .write
  .format("delta")
  .partitionBy("Country")
  .mode("append")
  .save(DataPath)
)

Perform a simple `count` query to verify the number of records and notice it is correct.

Should be `65535`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The changes to our files have been immediately reflected in the table that we've registered.

### Key Takeaways
With Delta Lake, you can easily append new data without schema-on-read issues.

Changes to Delta Lake files will immediately be reflected in registered Delta tables.



## Delta Lake Batch Operations - Upsert

To UPSERT means to "UPdate" and "inSERT". In other words, UPSERT is literally TWO operations. It is not supported in traditional data lakes, as running an UPDATE could invalidate data that is accessed by the subsequent INSERT operation.

Using Delta Lake, however, we can do UPSERTS. Delta Lake combines these operations to guarantee atomicity to
- INSERT a row 
- if the row already exists, UPDATE the row.

### Scenario
You have a small amount of batch data to write to your Delta table. This is currently staged in a JSON in a mounted blob store.

upsertDF = spark.read.format("json").load("/mnt/training/enb/commonfiles/upsert-data.json")
display(upsertDF)

We'll register this as a temporary view so that this table doesn't persist in DBFS (but we can still use SQL to query it).

In [ ]:
upsertDF.createOrReplaceTempView("upsert_data")

Included in this data are:
- Some new orders for customer 20993
- An update to a previous order correcting the country for customer 20993 to Iceland
- Corrections to some records for StockCode 22837 where the Description was incorrect

We can use UPSERT to simultaneously INSERT our new data and UPDATE our previous records.

In [ ]:
%sql
MERGE INTO customer_data_delta
USING upsert_data
ON customer_data_delta.InvoiceNo = upsert_data.InvoiceNo
  AND customer_data_delta.StockCode = upsert_data.StockCode
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED
  THEN INSERT *

Notice how this data is seamlessly incorporated into `customer_data_delta`.

In [ ]:
%sql
SELECT * FROM customer_data_delta WHERE CustomerID=20993

In [ ]:
%sql
SELECT DISTINCT(Description) 
FROM customer_data_delta 
WHERE StockCode = 22837

## Summary
In this Lesson, we:
- Saved files using Delta Lake
- Used Delta Lake to UPSERT data into existing Delta Lake tables

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/delta-batch.html#" target="_blank">Table Batch Read and Writes</a>


# Delta Lake Basics Lab

Delta Lake allows you to read, write and query data in data lakes in an efficient manner.

## In this lesson you:
* Create a new Delta Lake from aggregate data of an existing Delta Lake
* UPSERT records into a Delta lake
* Append new data to an existing Delta Lake

## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Analysts and Data Scientists

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater

## Datasets Used
We will use online retail datasets from `/mnt/training/online_retail`

Because we'll be calculating some aggregates in this notebook, we'll change our partitions after shuffle from the default `200` to `8` (which is a good number for the 8 node cluster we're currently working on).

In [ ]:
%python

sqlContext.setConf("spark.sql.shuffle.partitions", "8")

## What is a table? 
Before we continue, we need to address a semantic concern addressed by the [Databricks docs](https://docs.databricks.com/user-guide/tables.html#view-databases-and-tables):

> A Databricks table is a collection of structured data. Tables are equivalent to Apache Spark DataFrames.

Generally, the distinction between tables and DataFrames in Spark can be summarized by discussing scope and persistence:
- Tables are defined at the **workspace** level and **persist** between notebooks.
- DataFrames are defined at the **notebook** level and are **ephemeral**.

When we discuss **Delta tables**, we are always talking about collections of structured data that persist between notebooks. Importantly, we do not need to register a directory of files to Spark SQL in order to refer to them as a table. The directory of files itself _is_ the table; registering it with a useful name to Spark SQL just gives us easy accessing to querying these underlying data.

A **Delta Lake** can be thought of as a collection of one or many Delta tables. Generally, an entire elastic storage container will be dedicated to a single Delta Lake, and data will be enriched and cleaned as it is promoted through pre-defined logic.

<img alt="Best Practice" title="Best Practice" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-blue-ribbon.svg"/> To make Delta tables easily accessible, register them using Spark SQL. Use table ACLs to control access in workspaces shared by many diverse parties within an organization.

### Creating a new Delta table

You business intelligence team wants to create a dashboard to track the total number of orders made by customers globally. Many of your customers are international retailers, and have the same customer ID.

Because you batch process your data each day, you've decided to create a workflow that will update their numbers when you run your reports each night.

In this notebook, we'll start by transforming existing data stored in Delta to create a new Delta table for your BI team's dashboard. Then, we'll create processes to append new data to our full records as well as updating the Delta table for the BI team.

Note that because we registered a global table associated with this Delta table, you should already be able to query this data with SQL. To see all your currently registered tables, click the `Data` icon on the left navigation bar.

<img src=https://s3-us-west-2.amazonaws.com/files.training.databricks.com/images/adbcore/data-button.png width=100px>

Because we stored our data in Delta, our schema and partions are preserved. All we'll need to do is specify the format and the path.

In [ ]:
deltaDF = (spark.read
  .format("delta")
  .load(DeltaPath))

### Generate our BI table
We'll start out by just looking at our aggregate counts. Here, we group by both `"CustomerID"` and `"Country"`, as it is the combination of these two fields that is of interest to our BI team.

In [ ]:
customerCounts = (deltaDF.groupBy("CustomerID", "Country")
  .count()
  .withColumnRenamed("count", "total_orders"))

display(customerCounts)

Clicking on the names of the various columns will allow us to quickly sort on different fields. You may notice that we have a large number of entries that are `null` for both `"CustomerID"` and `"Country"`. While in production, we would like to explore _why_ we are seeing these missing values, for now we'll just leave them as is and save out this DataFrame as a new Delta table.

Here, the path is provided for you.

In [ ]:
CustomerCountsPath = userhome + "/delta/customer_counts/"

dbutils.fs.rm(CustomerCountsPath, True) #deletes Delta table if previously created

Here we'll write out our Delta table to the path provided above. Make sure the following settings are provided:
- `overwrite` (so that this code will work if you run it again)
- format as `delta`
- partition by `"Country"`
- save to `CustomerCountsPath`

In [ ]:
(customerCounts.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("Country")
  .save(CustomerCountsPath))

We'll also register this Delta table as a Spark SQL table.

In [ ]:
spark.sql("""
  DROP TABLE IF EXISTS customer_counts
""")

spark.sql("""
  CREATE TABLE customer_counts
  USING DELTA
  LOCATION '{}'
""".format(CustomerCountsPath))

Now our BI team can quickly query those data points they've expressed interest in.

In [ ]:
%sql

SELECT *
FROM customer_counts

Looking at your existing Delta table, you know that a large number of recent orders haven't been loaded in yet.

###  READ updated CSV data

Read the data into a DataFrame. We'll use the same schema that we used when creating our Delta table, which is supplied for you.

In [ ]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

inputSchema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", StringType(), True),
  StructField("UnitPrice", DoubleType(), True),
  StructField("CustomerID", IntegerType(), True),
  StructField("Country", StringType(), True)
])

Read data in newDataPath. Re-use inputSchema as defined above. We'll name our DataFrame newDataDF.

In [ ]:
newDataPath = "/mnt/training/online_retail/outdoor-products/outdoor-products-small.csv"
newDataDF = (spark
 .read
 .option("header", "true")
 .schema(inputSchema)
 .csv(newDataPath))

Let's do the same aggregate count as above, on "Country" and "CustomerID".

In [ ]:
newCustomerCounts = (newDataDF.groupBy("CustomerID", "Country").count().withColumnRenamed("count", "total_orders"))

display(newCustomerCounts)

### UPSERT new customer counts

Now that we've successfully loaded and aggregated our new data, we can upsert it in our existing Delta Lake.

First, we'll register it as a temp view.

In [ ]:
newCustomerCounts.createOrReplaceTempView("new_customer_counts")

And now we can merge these new counts into our existing data.

In [ ]:
%sql

MERGE INTO customer_counts
USING new_customer_counts
ON customer_counts.Country = new_customer_counts.Country
AND customer_counts.CustomerID = new_customer_counts.CustomerID
WHEN MATCHED THEN
  UPDATE SET total_orders = customer_counts.total_orders + new_customer_counts.total_orders
WHEN NOT MATCHED THEN
  INSERT *

We can write a simple SQL query to confirm that this has worked.

In [ ]:
%sql

SELECT SUM(total_orders) FROM customer_counts

### Update full records using append

We want to retain the full records being generated from our batch report in our existing non-aggregated Delta table.

In this case, we're assuming that the records we process at the end of each day are correct, and that batch processing will result in correct, stable records. We can safely write our table to the same file path using the append mode to insert these records.

**Note**: If our reports included changes to line items from previous days, we would want to write an UPSERT which would allow us to simultaneously update our changed records and insert new data.

In [ ]:
(newDataDF.write
  .format("delta")
  .mode("append")
  .save(DeltaPath))

Querying our table again shows that we've immediately updated.

In [ ]:
%sql
SELECT COUNT(*) FROM customer_data_delta

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/delta-batch.html#" target="_blank">Table Batch Read and Writes</a>


# <img src="https://files.training.databricks.com/images/DeltaLake-logo.png" width=80px> Managed Delta Lake

Delta Lake&reg; managed and queried via the Databricks platform includes additional features and optimizations.

These include:

- **Optimize**

- **Data skipping**

- **Z-Order**

- **Caching**

<img src="https://www.evernote.com/l/AAGv1SuWeRNJM4TI4bIOyGNPm0CTHa17PLwB/image.png" width=900px>

In [ ]:
spark.sql("""
    DROP TABLE IF EXISTS iot_data
  """)
spark.sql("""
    CREATE TABLE iot_data
    USING DELTA
    LOCATION '{}/delta/iot-events/'
  """.format(userhome))

Set up relevant paths.

iotPath = userhome + "/delta/iot-events/"

## SMALL FILE PROBLEM

Historical and new data is often written in very small files and directories.

This data may be spread across a data center or even across the world (that is, not co-located).

The result is that a query on this data may be very slow due to
* network latency
* volume of file metatadata

The solution is to compact many small files into one larger file.
Delta Lake has a mechanism for compacting small files.

### OPTIMIZE
Delta Lake supports the `OPTIMIZE` operation, which performs file compaction.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Small files are compacted together into new larger files up to 1GB.
Thus, at this point the number of files increases!

The 1GB size was determined by the Databricks optimization team as a trade-off between query speed and run-time performance when running Optimize.

`OPTIMIZE` is not run automatically because you must collect many small files first.

* Run `OPTIMIZE` more often if you want better end-user query performance
* Since `OPTIMIZE` is a time consuming step, run it less often if you want to optimize cost of compute hours
* To start with, run `OPTIMIZE` on a daily basis (preferably at night when spot prices are low), and determine the right frequency for your particular business case
* In the end, the frequency at which you run `OPTIMIZE` is a business decision

The easiest way to see what `OPTIMIZE` does is to perform a simple `count(*)` query before and after and compare the timing!

Take a look at the `iotPath + "/date=2018-06-01/" ` directory.

Notice, in particular files like `../delta/iot-events/date=2018-07-26/part-xxxx.snappy.parquet`. There are hundreds of small files!

In [ ]:
display(dbutils.fs.ls(iotPath + "/date=2016-07-26"))

CAUTION: Run this query. Notice it is very slow, due to the number of small files.

In [ ]:
%sql
SELECT * FROM iot_data where deviceId=92

### Data Skipping and ZORDER

Delta Lake uses two mechanisms to speed up queries.

<b>Data Skipping</b> is a performance optimization that aims at speeding up queries that contain filters (WHERE clauses).

For example, we have a data set that is partitioned by `date`.

A query using `WHERE date > 2016-07-26` would not access data that resides in partitions that correspond to dates prior to `2016-07-26`.

<b>ZOrdering</b> is a technique to colocate related information in the same set of files.

ZOrdering maps multidimensional data to one dimension while preserving locality of the data points.

Given a column that you want to perform ZORDER on, say `OrderColumn`, Delta
* takes existing parquet files within a partition
* maps the rows within the parquet files according to `OrderColumn` using the algorithm described <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">here</a>
* (in the case of only one column, the mapping above becomes a linear sort)
* rewrites the sorted data into new parquet files

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You cannot use the partition column also as a ZORDER column.

#### ZORDER Technical Overview

A brief example of how this algorithm works (refer to [this blog](https://databricks.com/blog/2018/07/31/processing-petabytes-of-data-in-seconds-with-databricks-delta.html) for more details):

![](https://files.training.databricks.com/images/adbcore/zorder.png)

Legend:
- Gray dot = data point e.g., chessboard square coordinates
- Gray box = data file; in this example, we aim for files of 4 points each
- Yellow box = data file that’s read for the given query
- Green dot = data point that passes the query’s filter and answers the query
- Red dot = data point that’s read, but doesn’t satisfy the filter; “false positive”

#### ZORDER example
In the image below, table `Students` has 4 columns:
* `gender` with 2 distinct values
* `Pass-Fail` with 2 distinct values
* `Class` with 4 distinct values
* `Student` with many distinct values

Suppose you wish to perform the following query:

```SELECT Name FROM Students WHERE gender = 'M' AND Pass_Fail = 'P' AND Class = 'Junior'```

```ORDER BY Gender, Pass_Fail```

The most effective way of performing that search is to order the data starting with the largest set, which is `Gender` in this case.

If you're searching for `gender = 'M'`, then you don't even have to look at students with `gender = 'F'`.

Note that this technique only works if all `gender = 'M'` values are co-located.


<div><img src="https://files.training.databricks.com/images/eLearning/Delta/zorder.png" style="height: 300px"/></div><br/>

#### ZORDER usage

With Delta Lake the notation is:

> `OPTIMIZE Students`<br>
`ZORDER BY Gender, Pass_Fail`

This will ensure all the data backing `Gender = 'M' ` is colocated, then data associated with `Pass_Fail = 'P' ` is colocated.

See References below for more details on the algorithms behind ZORDER.

Using ZORDER, you can order by multiple columns as a comma separated list; however, the effectiveness of locality drops.

In streaming, where incoming events are inherently ordered (more or less) by event time, use `ZORDER` to sort by a different column, say 'userID'.

In [ ]:
%sql
OPTIMIZE iot_data
ZORDER by (deviceId)

In [ ]:
%sql
SELECT * FROM iot_data WHERE deviceId=92

## VACUUM

To save on storage costs you should occasionally clean up invalid files using the `VACUUM` command.

Invalid files are small files compacted into a larger file with the `OPTIMIZE` command.

The  syntax of the `VACUUM` command is
>`VACUUM name-of-table RETAIN number-of HOURS;`

The `number-of` parameter is the <b>retention interval</b>, specified in hours.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> Databricks does not recommend you set a retention interval shorter than seven days because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table.

The scenario here is:
0. User A starts a query off uncompacted files, then
0. User B invokes a `VACUUM` command, which deletes the uncompacted files
0. User A's query fails because the underlying files have disappeared

Invalid files can also result from updates/upserts/deletions.

More details are provided here: <a href="https://docs.databricks.com/delta/optimizations.html#garbage-collection" target="_blank"> Garbage Collection</a>.

In [ ]:
len(dbutils.fs.ls(iotPath + "date=2016-07-26"))

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> In the example below we set off an immediate `VACUUM` operation with an override of the retention check so that all files are cleaned up immediately.

Do not do this in production!

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> If using Databricks Runtime 5.1, in order to use a retention time of 0 hours, the following flag must be set.

In [ ]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [ ]:
%sql

VACUUM iot_data RETAIN 0 HOURS;

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Notice how the directory looks vastly cleaned up!

In [ ]:
len(dbutils.fs.ls(iotPath + "date=2016-07-26"))

## Summary
Delta Lake offers key features that allow for query optimization and garbage collection, resulting in improved performance.

## Additional Topics & Resources

* <a href="https://docs.databricks.com/delta/optimizations.html#" target="_blank">Optimizing Performance and Cost</a>
* <a href="http://parquet.apache.org/documentation/latest/" target="_blank">Parquet Metadata</a>
* <a href="https://en.wikipedia.org/wiki/Z-order_curve" target="_blank">Z-Order Curve</a>
